# Missing Values

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('./data/melb_data.csv')
y = data['Price']

# Using only numerical predictors...
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Splitting the data into training and validation subsets
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0)

X_train.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
664,3,9.2,3104.0,3.0,2.0,2.0,368.0,177.0,2009.0,-37.78460,145.09350,7809.0
3270,2,10.5,3081.0,2.0,1.0,2.0,586.0,80.0,1955.0,-37.74350,145.04860,2947.0
3873,2,11.2,3145.0,2.0,1.0,1.0,348.0,NaN,NaN,-37.86720,145.04320,8801.0
13170,3,19.6,3076.0,3.0,1.0,1.0,521.0,NaN,NaN,-37.63854,145.05179,10926.0
1730,4,11.4,3163.0,3.0,2.0,2.0,687.0,237.0,1983.0,-37.89310,145.04790,7822.0


In [2]:
X_train.isnull().sum()

Rooms               0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                47
Landsize            0
BuildingArea     4843
YearBuilt        4042
Lattitude           0
Longtitude          0
Propertycount       0
dtype: int64

## Three Approaches...
1. **Drop Columns with Missing Values**
2. **Imputation**: This is the standard approach of filling in the missing values with some number (ex. mean value)
3. **An Extension to Imputation**: Imputed values may be systematically above or below their actual values or rows with missing values may be unique in some other way. In this case, the model would make better predictions by considering which values were originally missing. In this approach, the missing values are imputed, and additionally, for each column with missing entries in the original dataset, a new column is added that shows the location for the imputed entries.

In [3]:
# defining a function to measure quality of each approach:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_val, y_train, y_val):
    model = RandomForestRegressor(n_estimators = 10, random_state=0)
    model.fit(X_train, y_train)
    prediction = model.predict(X_val)
    return mean_absolute_error(y_val, prediction)

### Approach 1: Drop Columns with Missing Values

In [4]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_val = X_val.drop(cols_with_missing, axis=1)

print("MAE from Approach 1: ")
print(score_dataset(reduced_X_train, reduced_X_val, y_train, y_val))

MAE from Approach 1: 
187451.32209832387


### Approach 2: Imputation
Filling in the missing values with mean value along each column

In [5]:
from sklearn.impute import SimpleImputer

# imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_val = pd.DataFrame(my_imputer.transform(X_val))

imputed_X_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,3.0,9.2,3104.0,3.0,2.0,2.0,368.0,177.000000,2009.000000,-37.78460,145.09350,7809.0
1,2.0,10.5,3081.0,2.0,1.0,2.0,586.0,80.000000,1955.000000,-37.74350,145.04860,2947.0
2,2.0,11.2,3145.0,2.0,1.0,1.0,348.0,154.655601,1964.938304,-37.86720,145.04320,8801.0
3,3.0,19.6,3076.0,3.0,1.0,1.0,521.0,154.655601,1964.938304,-37.63854,145.05179,10926.0
4,4.0,11.4,3163.0,3.0,2.0,2.0,687.0,237.000000,1983.000000,-37.89310,145.04790,7822.0


In [6]:
# imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_val.columns = X_val.columns

print("MAE from Approach 2: ")
print(score_dataset(imputed_X_train, imputed_X_val, y_train, y_val))

MAE from Approach 2: 
183550.01819342168


### Approach 3: Extension to Imputation
Impute the missing values while also keeping track of which values were imputed

In [14]:
# Make copy to avoid changing original data
new_X_train = X_train.copy()
new_X_val = X_val.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    new_X_train[col + "_was_missing"] = new_X_train[col].isnull()
    new_X_val[col + "_was_missing"] = new_X_val[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_new_X_train = pd.DataFrame(my_imputer.fit_transform(new_X_train))
imputed_new_X_val = pd.DataFrame(my_imputer.transform(new_X_val))

# bringing back the column names
imputed_new_X_train.columns = new_X_train.columns
imputed_new_X_val.columns = new_X_val.columns

In [15]:
print("MAE from Approach 3: ")
print(score_dataset(imputed_new_X_train, imputed_new_X_val, y_train, y_val))

MAE from Approach 3: 
183554.2254688267


## Results
* **Approach 1** MAE: ```187451.32209832387```
* **Approach 2** MAE: ```183550.01819342168```
* **Approach 3** MAE: ```183554.2254688267```